## Домашнее задание : "Обучение с подкреплением"

ФИО: Чапаев Артем Юрьевич

In [1]:
!pip install setuptools
!pip install -v cmake gymnasium 'gym[toy_text]' gym[atari]
!pip install gym[accept-rom-license]
!pip install atari-py
!pip install stable-baselines3[extra] pyvirtualdisplay gym[atari] pyglet 

Using pip 23.2.1 from /opt/conda/lib/python3.10/site-packages/pip (python 3.10)
  Obtaining dependency information for cmake from https://files.pythonhosted.org/packages/09/30/df85689d18122becb9b6495cf6778f9ef629bdaa3ec86f49809ab5772e35/cmake-3.28.1-py2.py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 50.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 42.4 MB/s eta 0:00:00:00:0100:01
  changing mode of /opt/conda/bin/cmake to 755
  changing mode of /opt/conda/bin/cpack to 755
  changing mode of /opt/conda/bin/ctest to 755
  changing mode of /opt/conda/bin/ale-import-roms to 755
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 7.4 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... don

In [25]:
!pip install imageio[ffmpeg]

  Obtaining dependency information for imageio-ffmpeg from https://files.pythonhosted.org/packages/1a/98/3df1d8dd8f2c121b6c588b1e0d604f36592d56df9c41fb155ed546c6a5ed/imageio_ffmpeg-0.4.9-py3-none-manylinux2010_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 42.5 MB/s eta 0:00:00:00:0100:01


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import random

import tqdm
from tqdm.notebook import tqdm

import gymnasium as gym

# Задание 1

Обучите алгоритм Q-learning для сред FrozenLake-v1 и Blackjack-v1, в частности подберите оптимальную alpha. (2 балла)

In [ ]:
def initialize_q_table(state_space, action_space):
    Qtable = np.zeros((*state_space, action_space))
    return Qtable

In [ ]:
def greedy_policy(Qtable, state):
    # Exploitation: take the action with the highest state, action value
    action = np.argmax(Qtable[state][:])

    return action

def epsilon_greedy_policy(Qtable, state, epsilon):
    # Randomly generate a number between 0 and 1
    random_num = random.random()
    # if random_num > greater than epsilon --> exploitation
    if random_num > epsilon:
        # Take the action with the highest value given a state
        # np.argmax can be useful here
        action = greedy_policy(Qtable, state)
    # else --> exploration
    else:
        action = env.action_space.sample() # Take a random action

    return action

In [ ]:
def train(n_training_episodes, learning_rate, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
    for episode in tqdm(range(n_training_episodes)):
        # Reduce epsilon (because we need less and less exploration)
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate*episode)
        # Reset the environment
        state, info = env.reset()
        
        step = 0
        terminated = False
        truncated = False

        # repeat
        for step in range(max_steps):
            # Choose the action At using epsilon greedy policy
            action = epsilon_greedy_policy(Qtable, state, epsilon)

            # Take action At and observe Rt+1 and St+1
            # Take the action (a) and observe the outcome state(s') and reward (r)
            new_state, reward, terminated, truncated, info = env.step(action)

            # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
            Qtable[state][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])

            # If terminated or truncated finish the episode
            if terminated or truncated:
                break

            # Our next state is the new state
            state = new_state
    return Qtable

In [ ]:
def evaluate_agent(env, max_steps, n_eval_episodes, Q):
    """
    Evaluate the agent for ``n_eval_episodes`` episodes and returns average reward and std of reward.
    :param env: The evaluation environment
    :param n_eval_episodes: Number of episode to evaluate the agent
    :param Q: The Q-table
    :param seed: The evaluation seed array (for FrozenLake-v1)
    """
    episode_rewards = []
    for episode in tqdm(range(n_eval_episodes)):
        state, info = env.reset()

        truncated = False
        terminated = False
        total_rewards_ep = 0

        for step in range(max_steps):
            # Take the action (index) that have the maximum expected future reward given that state
            action = greedy_policy(Q, state)
            new_state, reward, terminated, truncated, info = env.step(action)
            total_rewards_ep += reward

            if terminated or truncated:
                break
            state = new_state
        episode_rewards.append(total_rewards_ep)

    return np.mean(episode_rewards), np.std(episode_rewards)

### FrozenLake

In [ ]:
env = gym.make("FrozenLake-v1")

env.reset()

In [ ]:
action_to_i = {
    'left': 0,
    'down': 1,
    'right': 2,
    'up': 3
}

In [ ]:
state_space = env.observation_space.n
print("There are ", f"{state_space}", " possible states")

action_space = env.action_space.n
print("There are ", action_space, " possible actions")

In [ ]:
Qtable_frozenlake = initialize_q_table([state_space], action_space)
print("Q-table shape: ", Qtable_frozenlake.shape)

In [ ]:
# Training parameters
n_training_episodes = 1_500_000   # Total training episodes
learning_rate = 0.2          # Learning rate

# Evaluation parameters
n_eval_episodes = 100        # Total number of test episodes

# Environment parameters
env_id = "FrozenLake-v1"     # Name of the environment
max_steps = 99               # Max steps per episode
gamma = 0.95                 # Discounting rate
eval_seed = []               # The evaluation seed of the environment

# Exploration parameters
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.05            # Minimum exploration probability
decay_rate = 0.0005            # Exponential decay rate for exploration prob

С помощью перебора по сетке были определены наилучшие гиперпараметры

In [ ]:
Qtable_frozenlake = initialize_q_table([state_space], action_space)
Qtable_frozenlake = train(n_training_episodes, learning_rate, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable_frozenlake)
evaluate_agent(env, max_steps, n_eval_episodes, Qtable_frozenlake)

### Blackjack

In [ ]:
env = gym.make("Blackjack-v1")

env.reset()

In [ ]:
action_to_i = {
    'stick':0,
    'hit':1
}

In [ ]:
state_space = [space.n for space in env.observation_space]
print("There are ", f"{state_space[0]}x{state_space[1]}x{state_space[2]}", " possible states")

action_space = env.action_space.n
print("There are ", action_space, " possible actions")

In [ ]:
Qtable_blackjack = initialize_q_table(state_space, action_space)
print("Q-table shape: ", Qtable_blackjack.shape)

In [ ]:
# Training parameters
n_training_episodes = 500_000   # Total training episodes

# Evaluation parameters
n_eval_episodes = 500        # Total number of test episodes

# Environment parameters
env_id = "Blackjack-v1"     # Name of the environment
max_steps = 99               # Max steps per episode
gamma = 0.95                 # Discounting rate

# Exploration parameters
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.05            # Minimum exploration probability 

С помощью перебора по сетке были определены наилучшие гиперпараметры (какие именно были результаты можно посмотреть в первом сохранении ноутбука)

In [ ]:
lr = 0.3
dr = 0.05

Qtable_blackjack = initialize_q_table(state_space, action_space)
Qtable_blackjack = train(n_training_episodes, lr, min_epsilon, max_epsilon, dr, env, max_steps, Qtable_blackjack)
evaluate_agent(env, max_steps, n_eval_episodes, Qtable_blackjack)

In [ ]:
lr = 0.5
dr = 0.25

Qtable_blackjack = initialize_q_table(state_space, action_space)
Qtable_blackjack = train(n_training_episodes, lr, min_epsilon, max_epsilon, dr, env, max_steps, Qtable_blackjack)
evaluate_agent(env, max_steps, n_eval_episodes, Qtable_blackjack)

# Задание 2

Обучите алгоритм Policy Gradients (или Actor Critic) для среды https://www.gymlibrary.dev/environments/atari/breakout/ . Продемонстрируйте, что для обученного агента растет время игры. (3 балла)

Для этого задания использована готовую модель Proximal Policy Optimization из библиотеки stable_baseline3. Модель реализована на основе алгоритма Policy Optimization

In [3]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback

In [4]:
env = make_atari_env('Breakout-v4', n_envs=4)
env = VecFrameStack(env, n_stack=4)

_ = env.reset()

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [5]:
model = PPO('CnnPolicy', env, verbose=1)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [6]:
def get_breakout_time(env, model):
    episodes = 10
    
    steps = 0
    for episode in range(episodes):
        state = env.reset()
        done = np.array([False] * 4)
        score = 0

        while not done.any():
            action , _ = model.predict(state)
            state, reward, done, info = env.step(action)
            score += reward
            steps += 1
        print(f"Episode: {episode} | Score: {score}")
    
    print(f'\nMean game time: {steps / episodes:0.1f} frames')

In [62]:
get_breakout_time(env, model)

Episode: 0 | Score: [0. 0. 0. 0.]
Episode: 1 | Score: [0. 0. 1. 0.]
Episode: 2 | Score: [0. 0. 0. 0.]
Episode: 3 | Score: [0. 0. 0. 0.]
Episode: 4 | Score: [0. 0. 1. 0.]
Episode: 5 | Score: [0. 1. 0. 0.]
Episode: 6 | Score: [0. 0. 0. 0.]
Episode: 7 | Score: [0. 0. 0. 0.]
Episode: 8 | Score: [0. 0. 0. 0.]
Episode: 9 | Score: [0. 0. 0. 0.]

Mean game time: 2.5 frames


In [8]:
model.learn(total_timesteps=100_000, progress_bar=True)

Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 276      |
|    ep_rew_mean     | 1.44     |
| time/              |          |
|    fps             | 147      |
|    iterations      | 1        |
|    time_elapsed    | 55       |
|    total_timesteps | 8192     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 307         |
|    ep_rew_mean          | 2.03        |
| time/                   |             |
|    fps                  | 84          |
|    iterations           | 2           |
|    time_elapsed         | 193         |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.016362276 |
|    clip_fraction        | 0.113       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38       |
|    explained_variance   | -0.00709    |
|    learning_rate        | 0.

In [9]:
model.save("ppo_model")

loaded_model = PPO.load("ppo_model")

In [79]:
get_breakout_time(env, loaded_model)

Episode: 0 | Score: [5. 4. 4. 1.]
Episode: 1 | Score: [0. 0. 0. 0.]
Episode: 2 | Score: [0. 0. 0. 1.]
Episode: 3 | Score: [0. 0. 0. 0.]
Episode: 4 | Score: [0. 0. 0. 0.]
Episode: 5 | Score: [1. 0. 0. 0.]
Episode: 6 | Score: [0. 1. 0. 0.]
Episode: 7 | Score: [0. 0. 0. 0.]
Episode: 8 | Score: [0. 0. 0. 0.]
Episode: 9 | Score: [0. 0. 0. 0.]

Mean game time: 8.0 frames


In [23]:
def get_game_frames(env, model):
    frames = []
    
    state = env.reset()
    done = np.array([False] * 4)
    while not done.any():
        frames.append(env.render(mode='rgb_array'))
        action, _ = model.predict(state)
        state, _, done, _ = env.step(action)
    
    return frames

In [45]:
import imageio
from IPython.display import Video

imageio.mimsave("breakout_game.mp4", get_game_frames(env, enhanced_model), format='FFMPEG')
Video("breakout_game.mp4", embed=True)

Можно заметить, что время игры увеличилось